Let's first save the results from our homepage's quickstart into a plot object. 

In [1]:
from distortions.geometry import Geometry, bind_metric, local_distortions, neighborhoods
from distortions.visualization import dplot
import anndata as ad
import numpy as np
import scanpy as sc

adata = ad.AnnData(np.random.poisson(2, size=(100, 5)))
sc.pp.neighbors(adata, n_neighbors=15)
sc.tl.umap(adata)

geom = Geometry(affinity_kwds={"radius": 2}, adjacency_kwds={"n_neighbors": 15})
_, Hvv, Hs = local_distortions(adata.obsm["X_umap"], adata.X, geom)
embedding = bind_metric(adata.obsm["X_umap"], Hvv, Hs)

N = neighborhoods(adata, outlier_factor=1)
plot = dplot(embedding)\
    .mapping(x="embedding_0", y="embedding_1")\
    .inter_edge_link(N=N)\
    .geom_ellipse()

If you print plot, you will see the figure. Hovering the mouse near a point highlights its neighbors. If you double click the figure, it freezes the interactivity.

In [2]:
plot

dplot(dataset=[{'embedding_0': 2.251615047454834, 'embedding_1': 4.680741786956787, 'x0': -0.9575956899353978,…

![](https://raw.githubusercontent.com/krisrs1128/distortions-data/main/figures/quickstart.gif)

To save a specific view, call the `save()` method on the plot object. By default, it will save a file `plot.svg` to the current notebook directory. You can specify your own path, but it must be an SVG file.

In [3]:
plot.save("output.svg")

One known limitation is that you have to at least hover over the SVG before the plot will properly save. This is because anywidget loads some utilities lazily, and we can't save until some interactions have been made. The first time you run `plot.save()` (before any interactions), it might save an empty svg file to the directory.

In [4]:
embedding = adata.obsm["X_umap"].copy()
radius = 3 * np.mean(adata.obsp["distances"].data)
geom = Geometry("brute", laplacian_method="geometric", laplacian_kwds={"scaling_epps": 5}, adjacency_kwds={"n_neighbors": 15}, affinity_kwds={"radius": radius})
H, _, _ = local_distortions(embedding, adata.X, geom)
metrics = {k: H[k] for k in range(len(H))}
embedding = bind_metric(embedding, Hvv, Hs)

plot = dplot(embedding)\
    .mapping(x="embedding_0", y="embedding_1")\
    .inter_isometry(metrics=metrics)\
    .geom_ellipse()
plot

dplot(dataset=[{'embedding_0': 2.251615047454834, 'embedding_1': 4.680741786956787, 'x0': -0.9575956899353978,…

In some cases, we don't need to save the visualization but do want to extract the coordinates after local correction. In this case, we can run the `.correct()` method on the plot object. If you interact with the scatterplot, freeze the view (escape or double click), and then rerun `.correct()`, you will see a new DataFrame giving the coordinates of the updated ellipses in the `embedding_0` and `embedding_1` columns. The new ellipse axes lengths and orientations are given by the remaining columns `s0`, ..., `y0`.  The `kernel` columns provide the relatively distance between the mouse position and each sample.

In [6]:
plot.correct()

,embedding_0,embedding_1,s0,s1,x0,y0,new_angle,kernel_transform,kernel_metric,_id
0,2.253500,4.691962,1.189334,0.521185,-0.957538,-0.287099,106.690346,0.001487,7.272600e-15,0
1,1.297185,5.729176,0.564175,0.108757,0.010630,0.996738,179.388973,0.003757,7.489159e-13,1
2,-0.040127,5.807181,0.469599,0.174629,-0.698313,-0.685908,134.486510,0.022907,6.307720e-09,2
3,0.179831,7.515899,1.478568,0.152504,-0.818625,0.573564,54.983286,0.000743,2.258193e-16,3
4,-0.551927,5.432266,0.396329,0.129073,-0.600575,-0.677469,138.443051,0.089386,5.706070e-06,4
...,...,...,...,...,...,...,...,...,...,...
95,3.052271,6.196152,1.107401,0.468267,-0.989551,0.144182,81.710090,0.000114,1.888404e-20,95
96,2.116971,4.225777,0.997708,0.388609,-0.888953,-0.454999,117.105059,0.002240,5.633209e-14,96
97,0.747469,7.569689,0.453943,0.272769,-0.930161,0.366900,68.473318,0.000427,1.420033e-17,97
98,-1.190607,5.257383,0.405093,0.068417,-0.885394,-0.266060,106.725458,0.188436,2.375840e-04,98
